In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, FloatSlider, IntSlider
import time  # Para medir o tempo de execução

# Função que simula o modelo SEIRS com beta sazonal
def simula_seirs(beta_base, beta_amplitude, sigma, gamma, omega, N, dt, days, initial_exposed, initial_infected):
    # Cálculo dos passos de tempo
    passos = int(days / dt) + 1
    tempo = np.linspace(0, days, passos)

    # Inicializa vetores apenas para I (pois só precisamos de I(t) no gráfico)
    I = np.zeros(passos)
    S, E, R = N - initial_exposed - initial_infected, initial_exposed, 0.0
    I[0] = initial_infected

    for t in range(passos - 1):
        # Beta sazonal: varia com cosseno
        beta = beta_base * (1 + beta_amplitude * np.cos(2 * np.pi * tempo[t] / 365))

        # Atualização das variáveis
        new_exposed = beta * S * I[t] / N
        new_infected = sigma * E
        new_recovered = gamma * I[t]
        new_susceptible = omega * R

        S += (-new_exposed + new_susceptible) * dt
        E += (new_exposed - new_infected) * dt
        I[t+1] = I[t] + (new_infected - new_recovered) * dt
        R += (new_recovered - new_susceptible) * dt

    return tempo, I / N  # Retorna I(t) como proporção da população total

# Função para criar a interface interativa
def interactive_seirs():
    # Define os botões deslizantes
    sliders = {
        "beta_base": FloatSlider(value=0.2, min=0.1, max=1.0, step=0.01, description='Beta Base'),
        "beta_amplitude": FloatSlider(value=0.1, min=0.0, max=1.0, step=0.01, description='Amplitude Beta'),
        "sigma": FloatSlider(value=1/7, min=0.01, max=1.0, step=0.01, description='Sigma'),
        "gamma": FloatSlider(value=1/5, min=0.01, max=1.0, step=0.01, description='Gamma'),
        "omega": FloatSlider(value=1/365, min=0.014, max=0.1, step=0.001, description='Omega'),
        "N": IntSlider(value=100000, min=1000, max=1000000, step=100, description='População Total (N)'),
        "initial_exposed": IntSlider(value=10, min=0, max=1000, step=1, description='Expostos Iniciais'),
        "initial_infected": IntSlider(value=0, min=0, max=1000, step=1, description='Infectados Iniciais'),
        "days": IntSlider(value=1825, min=30, max=3650, step=1, description='Dias de Simulação'),
        "dt": FloatSlider(value=1, min=0.5, max=1.5, step=0.1, description='Passo de Tempo (dt)')
    }

    # Função para atualizar o gráfico e medir o tempo de execução
    def update_plot(**kwargs):
        # Mede o tempo de execução da função simula_seirs
        start_time = time.process_time()  # Início da medição
        tempo, I_prop = simula_seirs(**kwargs)  # I_prop é a proporção de infectados
        end_time = time.process_time()  # Fim da medição

        # Calcula o tempo de CPU gasto
        cpu_time = end_time - start_time

        # Exibe o tempo de execução
        print(f"Tempo de CPU para gerar I(t): {cpu_time:.4f} segundos")

        # Converte tempo de dias para anos
        tempo_em_anos = tempo / 365

        # Plota o gráfico
        plt.figure(figsize=(10, 6))
        plt.plot(tempo_em_anos, I_prop * 100, lw=2, color='blue')  # Multiplica por 100 para porcentagem
        plt.title("Evolução de Infectados ($t$ em Anos)")
        plt.xlabel("Anos")
        plt.ylabel("Proporção de Infectados (%)")  # Rótulo em porcentagem
        plt.ylim(0, 14)  # Limita o eixo Y entre 0% e 20%
        plt.grid()
        plt.show()

    # Cria a interface interativa
    interact(update_plot, **sliders)

# Executa a interface interativa
interactive_seirs()

interactive(children=(FloatSlider(value=0.2, description='Beta Base', max=1.0, min=0.1, step=0.01), FloatSlide…